In [ ]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_groq import ChatGroq
from langchain_cohere import CohereEmbeddings
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = (
    "pcsk_4UTPKe_5mkXoweEw83pjDm5gebcCdjRA76kvzZ1fysbaNsnugm2ds7hyHJQ5YFtJ78oMk5"
)


In [2]:
def read_doc(directory):
    file = PyPDFDirectoryLoader(directory)
    doc = file.load()
    return doc


In [ ]:
doc = read_doc("documents/")
doc


In [4]:
def chunk_data(docs, chunk_size=810, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    doc = text_splitter.split_documents(docs)
    return doc


In [5]:
documents = chunk_data(docs=doc)
# documents


In [ ]:
embeddings = CohereEmbeddings(
    cohere_api_key="fyYH6Yv6trfc81mWAtEjMqU8Uvnl5f77qIQjJT5g",
    model="embed-english-v3.0",
)


In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    doc, index_name="pdf-talker", embedding=embeddings
)
vectorstore


In [9]:
def retrieve_query(query, k=2):
    matching_results = vectorstore.similarity_search(query=query, k=k)
    return matching_results


In [10]:
from langchain.chains.conversational_retrieval.base import ChatVectorDBChain

os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(model="llama3-8b-8192")


In [ ]:
chain = ChatVectorDBChain.from_llm(llm=llm, vectorstore=vectorstore, chain_type="stuff")


def retrieve_answer(query):


    # doc_search = retrieve_query(query=query)


    # print(doc_search)
    chat_history = []
    query = {"question": query, "chat_history": chat_history}


    response = chain.invoke(query)


    return response


In [ ]:
query = "How much the agriculture target will be increased by how many crores?"

retrieve_answer(query=query)
